In [1]:
from __future__ import print_function
import numpy as np
import itertools as it
import sympy


## Denominator: 6D

In [2]:
dim = 6

# Generate all possible iterations (only upper triangular matrices)
assert dim <= 6, "Larger dimension take too long."
red_dim = dim * (dim - 1) / 2
entries_all = [a for  a in it.product([True, False], repeat = red_dim)]
Ps_all = []
for entries in entries_all:
    P = np.zeros((dim, dim), dtype=bool)
    for i in range(dim - 1):
        si = sum(range(dim - 1 - i, dim - 1))
        P[i, i+1:] = entries[i + si: si + dim - 1]
    Ps_all.append(P)

# Throw away inconsistent ones
Ps = []
for P in Ps_all:
    append_cond = True
    for x in range(dim-1):
        for y in range(x + 1, dim):
            for z in range(y + 1, dim):
#                 print(x, y, z)
                if P[x, y] or P[x, z]:
#                     print(P[x, y], P[x, z], (P[x, y] and P[x, z]), P[y, z])
                    if (P[x, y] and P[x, z]) != P[y, z]:
                        append_cond *= False
    if append_cond:
        np.fill_diagonal(P, True)
        Ps.append(P)
del Ps_all

# Calculate multiplicity
m_Ps = np.zeros(len(Ps), dtype=int)

for j, P in enumerate(Ps):
    indices = set(range(dim))

    # Initial clique containing the first index
    cliques = [set([y for y, P_0_y in enumerate(P[0]) if P_0_y])]
    
    # Form other cliques
    while set.union(*cliques) != indices:
        idx = min(indices - set.union(*cliques))
        P_idx = P[idx, idx: ]
        cliques.append(set([y + idx for y, P_idx_y in enumerate(P_idx) if P_idx_y]))

    # Collapse cliques
    m_Ps[j] = len(np.sort([len(clique) for clique in cliques]))
    
# For denominator: collapse 6d to 4d
dim_R = 4
Rs = np.zeros((len(Ps), dim_R, dim_R), dtype=bool)
for j, P in enumerate(Ps):
    R = np.zeros((dim_R, dim_R), dtype=bool)
    
    R[0, 1] = P[0, 4] and P[1, 4] #R[1, 1]
    R[0, 2] = P[0, 2] and P[4, 5] #R[0, 2]
    R[0, 3] = P[0, 5] and P[3, 4] #R[0, 3]
    R[1, 2] = P[2, 4] and P[1, 5] #R[1, 2]
    R[1, 3] = P[4, 5] and P[1, 3] #R[1, 3]
    R[2, 3] = P[2, 5] and P[3, 5] #R[2, 3]

    np.fill_diagonal(R, True)
    Rs[j] = R
    
# Project to clique lengths
sorted_clique_lens = []

for R in Rs:
    indices = set(range(dim_R))

    # Initial clique containing the first index
    cliques = [set([y for y, R_0_y in enumerate(R[0]) if R_0_y])]
    
    # Form other cliques
    while set.union(*cliques) != indices:
        idx = min(indices - set.union(*cliques))
        R_idx = R[idx, idx: ]
        cliques.append(set([y + idx for y, R_idx_y in enumerate(R_idx) if R_idx_y]))

    # Collapse cliques
    sorted_clique_lens.append(tuple(np.sort([len(clique) for clique in cliques])[::-1]))
    
unique_scls = set(sorted_clique_lens)
list_scl_m_c = []
for unique_scl in unique_scls:
    # Indices where the unique sorted clique length equals that of P in the list Ps
    idx = [j for j, scl in enumerate(sorted_clique_lens) if scl == unique_scl]
    
    # Get the number of appearence of multiplicity factors S, S-1, S-2, ..., S-5
    unique_m_P, counts = np.unique(m_Ps[idx], return_counts=True)
    m_factors = np.zeros(dim, dtype=int)
    m_factors[unique_m_P - 1] = counts
    
    # Append to a list containing the sored clique lengths and corresponding multiplicity factors
    list_scl_m_c.append([unique_scl, m_factors])
#     print(unique_scl, m_factors)

# Print the results in a nice way (to understand what's going on)
names                           = ['X^4    ','X^3 Y  ', 'X^2 Y^2', 'X^2 Y Z', 'X Y Z U',    ]
corresponding_clique_lengths    = [(4,),     (3, 1),    (2, 2),    (2, 1, 1), (1, 1, 1, 1), ]

print(" "*11 + "Multipl. number m_P")
m_p_str_title = ""
for i in range(dim):
    m_p_str_title += "%3d"%(i+1)
print("Cliques  " + m_p_str_title + " "*5 + " tuple")
print("-"*50)

m_factors_all = []
for name, cl in zip(names, corresponding_clique_lengths):
    for scl_m_c in list_scl_m_c:
        if scl_m_c[0] == cl:
            m_str = ""
            for m_c in scl_m_c[1]:
                m_str += "%3d"%m_c
            print(name + "  " + m_str + " "*5, scl_m_c[0])
            m_factors_all.append(scl_m_c[1])
m_factors_all = np.asarray(m_factors_all)

           Multipl. number m_P
Cliques    1  2  3  4  5  6      tuple
--------------------------------------------------
X^4        1  0  0  0  0  0      (4,)
X^3 Y      0  4  0  0  0  0      (3, 1)
X^2 Y^2    0  5  1  0  0  0      (2, 2)
X^2 Y Z    0 20 34  6  0  0      (2, 1, 1)
X Y Z U    0  2 55 59 15  1      (1, 1, 1, 1)


To get the multiplicity of each clique, take $\frac{S!}{(S - m_P)!} = S (S-1) (S-2)...(S - (m_P - 1))$ times the factor shown in the list.

In [3]:
# Symbolic calculations
S = sympy.symbols('S')
j = sympy.symbols('j', integer=True)

# Check-sum
sum_S = 0
for i in range(dim):
    prod_S = sympy.product(S-j, (j, 0, i))
    sum_S += prod_S * np.sum(m_factors_all, axis=0)[i]
print("Checksum == S**%d"%dim)
print("Checksum = ", sympy.expand(sum_S)) # == S**6???

# Calculate factors for expected value
a = []
for k in range(len(m_factors_all)):
    sum_S = 0
    for i in range(dim):
        prod_S = sympy.product(S-j, (j, 0, i))
        sum_S += prod_S * m_factors_all[k, i]
    a.append(sympy.expand(sum_S))
print("\nMultiplicity factors:")
for i, ai in enumerate(a):
    print("a_%d = "%i, ai)

# Calculate specific terms
mu3_term = 4 * a[0] + 1 * a[1]
mu2_term = 6 * a[0] + 3 * a[1] + 2 * a[2] + 1 * a[3]
mu1_term = sum(a)
print("\nTerms to multiply with:")
print("mu3: ", mu3_term)
print("mu2: ", mu2_term)
print("mu1: ", mu1_term)

# Print out the entire term
mu4, mu3, mu2, mu = sympy.symbols('mu4 mu3 mu2 mu')
E_d_sq = S**2 * (mu4 * a[0] + 
                 mu3 * mu * (4*a[0] + a[1]) + 
                 mu2 * mu**2 * (6*a[0] + 3*a[1] + 2*a[2] + a[3]) + 
                 mu2**2 * a[2] + 
                 mu**4 * sum(a))
print("\nE(d^2):")
print(sympy.expand(E_d_sq))


Checksum == S**6
Checksum =  S**6

Multiplicity factors:
a_0 =  S
a_1 =  4*S**2 - 4*S
a_2 =  S**3 + 2*S**2 - 3*S
a_3 =  6*S**4 - 2*S**3 - 16*S**2 + 12*S
a_4 =  S**6 - 6*S**4 + S**3 + 10*S**2 - 6*S

Terms to multiply with:
mu3:  4*S**2
mu2:  6*S**4
mu1:  S**6

E(d^2):
S**8*mu**4 + 6*S**6*mu**2*mu2 + S**5*mu2**2 + 4*S**4*mu*mu3 + 2*S**4*mu2**2 - 3*S**3*mu2**2 + S**3*mu4


##  Numerator

Reduced to 4d, but running over $M = S^2$ entries. 

In [4]:
dim = 4

# Generate all possible iterations (only upper triangular matrices)
assert dim <= 6, "Larger dimension take too long."
red_dim = dim * (dim - 1) / 2
entries_all = [a for  a in it.product([True, False], repeat = red_dim)]
Ps_all = []
for entries in entries_all:
    P = np.zeros((dim, dim), dtype=bool)
    for i in range(dim - 1):
        si = sum(range(dim - 1 - i, dim - 1))
        P[i, i+1:] = entries[i + si: si + dim - 1]
    Ps_all.append(P)

# Throw away inconsistent ones
Ps = []
for P in Ps_all:
    append_cond = True
    for x in range(dim-1):
        for y in range(x + 1, dim):
            for z in range(y + 1, dim):
#                 print(x, y, z)
                if P[x, y] or P[x, z]:
#                     print(P[x, y], P[x, z], (P[x, y] and P[x, z]), P[y, z])
                    if (P[x, y] and P[x, z]) != P[y, z]:
                        append_cond *= False
    if append_cond:
        np.fill_diagonal(P, True)
        Ps.append(P)
del Ps_all

### Calculate multiplicity
sorted_clique_lens = []
m_Ps = np.zeros(len(Ps), dtype=int)

for j, P in enumerate(Ps):
    indices = set(range(dim))
    # Initial clique containing the first index
    cliques = [set([y for y, P_0_y in enumerate(P[0]) if P_0_y])]
    
    # Form other cliques
    while set.union(*cliques) != indices:
        idx = min(indices - set.union(*cliques))
        P_idx = P[idx, idx: ]
        cliques.append(set([y + idx for y, P_idx_y in enumerate(P_idx) if P_idx_y]))

    # Collapse cliques
    sorted_clique_len = np.sort([len(clique) for clique in cliques])[::-1]
    sorted_clique_lens.append(tuple(sorted_clique_len))
    m_Ps[j] = len(sorted_clique_len)
    
    
### Get unique sorted cliques
unique_scls = set(sorted_clique_lens)
list_scl_m_c = []
for unique_scl in unique_scls:
    # Indices where the unique sorted clique length equals that of P in the list Ps
    idx = [j for j, scl in enumerate(sorted_clique_lens) if scl == unique_scl]
    
    # Get the number of appearence of multiplicity factors S, S-1, S-2, ..., S-5
    unique_m_P, counts = np.unique(m_Ps[idx], return_counts=True)
    m_factors = np.zeros(dim, dtype=int)
    m_factors[unique_m_P - 1] = counts
    
    # Append to a list containing the sored clique lengths and corresponding multiplicity factors
    list_scl_m_c.append([unique_scl, m_factors])
    # print(unique_scl, m_factors)

### Print the results in a nice way (to understand what's going on)
names                           = ['X^4    ','X^3 Y  ', 'X^2 Y^2', 'X^2 Y Z', 'X Y Z U',    ]
corresponding_clique_lengths    = [(4,),     (3, 1),    (2, 2),    (2, 1, 1), (1, 1, 1, 1), ]

print(" "*11 + "Multipl. number m_P")
m_p_str_title = ""
for i in range(dim):
    m_p_str_title += "%3d"%(i+1)
print("Cliques  " + m_p_str_title + " "*5 + " tuple")
print("-"*50)

m_factors_all = []
for name, cl in zip(names, corresponding_clique_lengths):
    for scl_m_c in list_scl_m_c:
        if scl_m_c[0] == cl:
            m_str = ""
            for m_c in scl_m_c[1]:
                m_str += "%3d"%m_c
            print(name + "  " + m_str + " "*5, scl_m_c[0])
            m_factors_all.append(scl_m_c[1])
m_factors_all = np.asarray(m_factors_all)

           Multipl. number m_P
Cliques    1  2  3  4      tuple
--------------------------------------------------
X^4        1  0  0  0      (4,)
X^3 Y      0  4  0  0      (3, 1)
X^2 Y^2    0  3  0  0      (2, 2)
X^2 Y Z    0  0  6  0      (2, 1, 1)
X Y Z U    0  0  0  1      (1, 1, 1, 1)


In [5]:
# Symbolic calculations
S = sympy.symbols('S')
M = S**2
j = sympy.symbols('j', integer=True)

# Check-sum
sum_M = 0
for i in range(dim):
    prod_M = sympy.product(M-j, (j, 0, i))
    sum_M += prod_M * np.sum(m_factors_all, axis=0)[i]
    
print("Checksum == M**%d"%dim)
print("Checksum = ", sympy.expand(sum_M)) # == M**6???

# Calculate factors for expected value
a = []
for k in range(len(m_factors_all)):
    sum_M = 0
    for i in range(dim):
        prod_M = sympy.product(M-j, (j, 0, i))
        sum_M += prod_M * m_factors_all[k, i]
    a.append(sympy.expand(sum_M))
    
print("\nMultiplicity factors:")
for i, ai in enumerate(a):
    print("a_%d = "%i, ai)

# Calculate specific terms
mu3_term = 4 * a[0] + 1 * a[1]
mu2_term = 6 * a[0] + 3 * a[1] + 2 * a[2] + 1 * a[3]
mu1_term = sum(a)

print("\nTerms to multiply with:")
print("mu3: ", mu3_term)
print("mu2: ", mu2_term)
print("mu1: ", mu1_term)

# Print out the entire term
mu4, mu3, mu2, mu = sympy.symbols('mu4 mu3 mu2 mu')
E_n_sq = (mu4 * a[0] + 
          mu3 * mu * (4*a[0] + a[1]) + 
          mu2 * mu**2 * (6*a[0] + 3*a[1] + 2*a[2] + a[3]) + 
          mu2**2 * a[2] + 
          mu**4 * sum(a))
print("E(n^2):")
print(sympy.expand(E_n_sq))


Checksum == M**4
Checksum =  S**8

Multiplicity factors:
a_0 =  S**2
a_1 =  4*S**4 - 4*S**2
a_2 =  3*S**4 - 3*S**2
a_3 =  6*S**6 - 18*S**4 + 12*S**2
a_4 =  S**8 - 6*S**6 + 11*S**4 - 6*S**2

Terms to multiply with:
mu3:  4*S**4
mu2:  6*S**6
mu1:  S**8
E(n^2):
S**8*mu**4 + 6*S**6*mu**2*mu2 + 4*S**4*mu*mu3 + 3*S**4*mu2**2 - 3*S**2*mu2**2 + S**2*mu4


## Putting numerator and denominator together

In [6]:
# Putting everything together

print("E(n^2) / S**2:")
print(sympy.expand(E_n_sq / S**2))

print("\nE(d^2) / S**2:")
print(sympy.expand(E_d_sq / S**2))

print("\n(E(n^2) - E(d^2)) / S**2:")
print(sympy.expand((E_n_sq - E_d_sq) / S**2))

Var_err = E_n_sq / E_d_sq - 1
sympy.simplify(Var_err)

E(n^2) / S**2:
S**6*mu**4 + 6*S**4*mu**2*mu2 + 4*S**2*mu*mu3 + 3*S**2*mu2**2 - 3*mu2**2 + mu4

E(d^2) / S**2:
S**6*mu**4 + 6*S**4*mu**2*mu2 + S**3*mu2**2 + 4*S**2*mu*mu3 + 2*S**2*mu2**2 - 3*S*mu2**2 + S*mu4

(E(n^2) - E(d^2)) / S**2:
-S**3*mu2**2 + S**2*mu2**2 + 3*S*mu2**2 - S*mu4 - 3*mu2**2 + mu4


(-S**3*mu2**2 + S**2*mu2**2 + 3*S*mu2**2 - S*mu4 - 3*mu2**2 + mu4)/(S*(S**5*mu**4 + 6*S**3*mu**2*mu2 + S**2*mu2**2 + 4*S*mu*mu3 + 2*S*mu2**2 - 3*mu2**2 + mu4))